## 环境配置

### 本地安装whl

前往网站下载对应的版本↓  
**pytorch**:https://download.pytorch.org/whl/torch_stable.html  
**pytorch-geometric**:https://pytorch-geometric.com/whl/  
在对应路径下打开cmd，使用pip install进行本地安装。  

### 配置时遇到的问题

#### 下载pytorch时网络不稳定使得下载失败

前往下载对应的whl文件，再本地安装后解决。

#### 导入时torch_cluster、torch_geometric、torch_scatter、torch_sparse、torch_spline_conv报错
OSError: [WinError 127] 找不到指定的程序。

发现是下载的版本与pytorch版本不一致。前往[https://pytorch-geometric.com/whl/](https://pytorch-geometric.com/whl/)下载对应的whl文件后解决。

### 配置完成

In [ ]:
import torch
import torch_cluster
import torch_geometric
import torch_scatter
import torch_sparse
import torch_spline_conv

In [23]:
print('cuda: \t\t\t',torch.cuda.is_available())
print('version: \t\t', torch.__version__)
print('cuda.version: \t\t', torch.version.cuda)
print('torch_cluster: \t\t', torch_cluster.__version__)
print('torch_geometric:\t', torch_geometric.__version__)
print('torch_scatter:\t\t', torch_scatter.__version__)
print('torch_sparse:\t\t', torch_sparse.__version__)
print('torch_spline_conv:\t', torch_spline_conv.__version__)

cuda: 			 True
version: 		 1.8.1+cu101
cuda.version: 		 10.1
torch_cluster: 		 1.5.9
torch_geometric:	 1.7.0
torch_scatter:		 2.0.7
torch_sparse:		 0.6.9
torch_spline_conv:	 1.2.1


## PyG

### Data对象
官方文档:[torch_geometric.data.Data](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data)

In [18]:
class Data(object):

    def __init__(self, x=None, edge_index=None, edge_attr=None, y=None, **kwargs):
        r"""
        Args:
            x (Tensor, optional): 节点属性矩阵，大小为`[num_nodes, num_node_features]`
            edge_index (LongTensor, optional): 边索引矩阵，大小为`[2, num_edges]`，第0行为尾节点，第1行为头节点，头指向尾
            edge_attr (Tensor, optional): 边属性矩阵，大小为`[num_edges, num_edge_features]`
            y (Tensor, optional): 节点或图的标签，任意大小（，其实也可以是边的标签）
        """
        self.x = x
        self.edge_index = edge_index
        self.edge_attr = edge_attr
        self.y = y

        for key, item in kwargs.items():
            if key == 'num_nodes':
                self.__num_nodes__ = item
            else:
                self[key] = item

In [44]:
Data()

In [31]:
from torch_geometric.datasets import KarateClub

dataset = KarateClub()
data = dataset[0]  # Get the first graph object.
print(data)
print('==============================================================')

# 获取图的一些信息
print(f'Number of nodes: {data.num_nodes}') # 节点数量
print(f'Number of edges: {data.num_edges}') # 边数量
print(f'Number of node features: {data.num_node_features}') # 节点属性的维度
print(f'Number of node features: {data.num_features}') # 同样是节点属性的维度
print(f'Number of edge features: {data.num_edge_features}') # 边属性的维度
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}') # 平均节点度
print(f'if edge indices are ordered and do not contain duplicate entries.: {data.is_coalesced()}') # 是否边是有序的同时不含有重复的边
print(f'Number of training nodes: {data.train_mask.sum()}') # 用作训练集的节点
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}') # 用作训练集的节点数占比
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}') # 此图是否包含孤立的节点
print(f'Contains self-loops: {data.contains_self_loops()}')  # 此图是否包含自环的边
print(f'Is undirected: {data.is_undirected()}')  # 此图是否是无向图

Data(edge_index=[2, 156], train_mask=[34], x=[34, 34], y=[34])
Number of nodes: 34
Number of edges: 156
Number of node features: 34
Number of node features: 34
Number of edge features: 0
Average node degree: 4.59
if edge indices are ordered and do not contain duplicate entries.: True
Number of training nodes: 4
Training node label rate: 0.12
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


### Dataset对象
官方文档:[torch_geometric.datasets](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#)

In [3]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/dataset/Cora', name='Cora')
print(dataset)
print('num_classes:', dataset.num_classes)
print('num_node_features:', dataset.num_node_features)

data = dataset[0]
print(data)
print('is_undirected:', data.is_undirected())
print('train_mask:', data.train_mask.sum().item())
print('val_mask:', data.val_mask.sum().item())
print('test_mask:', data.test_mask.sum().item())

## 作业
&emsp;&emsp;请通过继承Data类实现一个类，专门用于表示“机构-作者-论文”的网络。该网络包含“机构“、”作者“和”论文”三类节点，以及“作者-机构“和“作者-论文“两类边。  
&emsp;&emsp;对要实现的类的要求：  
- 1）用不同的属性存储不同节点的属性；  
- 2）用不同的属性存储不同的边（边没有属性）；  
- 3）逐一实现获取不同节点数量的方法。 

In [24]:
# 不太理解题目的意思？
class task1_Data(Data):
    def __init__(self, organization_x=None, author_x=None, article_x=None, 
                 organization_author_index=None, author_article_index=None):
        super(Data,self).__init__()
        self.organization_x = organization_x # 节点1:机构
        self.author_x = author_x # 节点2:作者
        self.article_x = article_x # 节点3:论文
        self.organization_author_index = organization_author_index # 边1:机构-作者
        self.author_article_index = author_article_index # 边2:作者-论文
    
    def get_organization_num():
        return organization_x.shape[1]
    
    def get_author_num():
        return author_x.shape[1]
    
    def get_article_num():
        return article_x.shape[1]